In [1]:
import nest_asyncio
nest_asyncio.apply()

import sys
sys.path.append("../")

from desci_sense.shared_functions.init import init_multi_chain_parser_config
from desci_sense.shared_functions.parsers.multi_chain_parser import MultiChainParser

from desci_sense.shared_functions.dataloaders import (
    scrape_post,
    convert_text_to_ref_post,
)

In [8]:
config = init_multi_chain_parser_config(llm_type="google/gemma-7b-it",
                                        post_process_type="combined")

In [9]:
multi_chain_parser = MultiChainParser(config)

2024-04-16 10:20:55.534 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:64 - Initializing MultiChainParser. PostProcessType=combined
2024-04-16 10:20:55.537 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:71 - Initializing post parsers...
2024-04-16 10:20:55.538 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:24 - Initializing parser chain 'refs_tagger' 
2024-04-16 10:20:55.580 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:24 - Initializing parser chain 'topics' 
2024-04-16 10:20:55.609 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:24 - Initializing parser chain 'keywords' 


In [10]:
# example raw text posts
TEST_POST_TEXT_W_REF = """
I really liked this paper!
https://arxiv.org/abs/2402.04607
"""

TEST_POST_2_TEXT_W_REF = """
I really disagree with this paper!
https://arxiv.org/abs/2402.04607
"""

TEXT_POSTS = [TEST_POST_TEXT_W_REF, TEST_POST_2_TEXT_W_REF]

In [11]:
# convert raw text to RefPost format for input to the parser
inputs = [convert_text_to_ref_post(p) for p in TEXT_POSTS]

In [12]:
# or scrape posts by url
urls = [
        "https://mastodon.social/@psmaldino@qoto.org/111405098400404613",
        "https://mastodon.social/@UlrikeHahn@fediscience.org/111732713776994953",
        "https://mastodon.social/@ronent/111687038322549430",
    ]
posts = [scrape_post(url) for url in urls]

In [13]:
# batch process
results = multi_chain_parser.batch_process_ref_posts(inputs)

2024-04-16 10:21:13.659 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation:54 - fetching citoid data for: https://arxiv.org/abs/2402.04607
2024-04-16 10:21:16.077 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:220 - Processing 2 posts with parsers: ['refs_tagger', 'topics', 'keywords']
2024-04-16 10:21:16.078 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:222 - Instantiating prompts...
2024-04-16 10:21:16.079 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:232 - Invoking parallel chain...
2024-04-16 10:21:19.645 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:242 - Post processing 2 results...
2024-04-16 10:21:19.647 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:254 - Done!


In [14]:
# single process
result = multi_chain_parser.process_ref_post(posts[0])

2024-04-16 10:21:28.353 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation:54 - fetching citoid data for: https://royalsocietypublishing.org/doi/10.1098/rstb.2022.0267
2024-04-16 10:21:29.502 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:173 - Processing post with parsers: ['refs_tagger', 'topics', 'keywords']
2024-04-16 10:21:29.503 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:175 - Instantiating prompts...
2024-04-16 10:21:29.504 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:180 - Invoking parallel chain...


In [16]:
result.filter_classification

<SciFilterClassfication.RESEARCH: 'research'>

In [18]:
result.reference_tagger

['recommendation', 'reading']

In [19]:
result.topics

['economics', 'research']